In [1]:
# -*- coding: utf-8 -*-
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd, numpy as np
import os
import re
from google.cloud import bigquery
from pandas.compat._optional import import_optional_dependency
pandas_gbq = import_optional_dependency("pandas_gbq", extra="")
pd.options.display.max_columns = None
import sys
sys.path.append('/Jupyter lab/Keyword classification')
from utils import *
from tqdm import tqdm
import time
import Levenshtein

In [2]:
sql = """
SELECT distinct *
FROM `spmena-sandbox-vn-apac-dv.vn_search_dashboard.search_keywords_combined_test`
"""
project_id = "spmena-sandbox-vn-apac-dv"
df_keyword = pd.read_gbq(sql,project_id=project_id, use_bqstorage_api=True) 
print(len(df_keyword))
df_keyword.head(2)

C:\Users\gianhubinh.le\AppData\Local\anaconda3\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


143353


,keyword
0,bộ sản phẩm trị mụn la roche posay
1,toner skin1004


### 1. Loading and transform the first unnamed row into Loreal/Competitor

In [3]:
keyword_type_df_brand = pd.read_excel("C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- VNCDO - Data work-space/Media Dashboard/Other/EMT Keyword.xlsx", sheet_name = 'Brand')
competitor_column_index = keyword_type_df_brand.columns.get_loc('Competitor')
column_name_mapping  = {}

for i in range(len(keyword_type_df_brand.columns)):
    if keyword_type_df_brand.columns.get_loc(keyword_type_df_brand.columns[i]) < competitor_column_index:
        column_name_mapping[keyword_type_df_brand.columns[i]] = "Branded"
    else:
        column_name_mapping[keyword_type_df_brand.columns[i]] = "Competitor"

for old_name, new_name in column_name_mapping.items():
    keyword_type_df_brand.rename(columns={old_name: new_name}, inplace=True)
keyword_type_df_brand.head(2)
# keyword_type_df_brand.iloc[:, 0]

,Branded,Branded,Branded,Branded,Branded,Branded,Branded,Branded,Branded,Branded,Branded,Branded,Branded,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor,Competitor
0,La Roche Posay,Skinceuticals,Vichy,Cerave,Garnier,Maybelline,L'Oreal Paris,Kiehl's,Lancome,shu uemura,Yves Saint Laurent,Kerastase,L'Oreal Professionnel,Perfect Diary,Black Rouge,3CE,Mac,Focallure,Merzy,Romand,Bbia,Innisfree,Lemonade,Bobbi Brown,O.Two.O,Zeesea,Sace Lady,Kimuse,PinkFlash,Colorkey,Gilaa,Bioderma,Ordinary,Anessa,Paula's Choice,Laneige,Klairs,Simple,Dr.Jart+,Estee Lauder,Skin1004,Shiseido,Clé de Peau,Chacott,Obagi,Clinique,Missha,Murad,Cocoon,Senka,Eucerin,Cetaphil,Neutrogena,Hiruscar,Srv,Dermarium
1,la roche posay,skinceuticals,vichy,Cerave,garnier,maybelline,loreal,kiehl's,lancone,shu,ysl,kérastase,professionnel,perfect dỉay,blackrouge,3ce,MAC,focallure,merzy,romand,bbia,innisfree,lemondade,bobbi,otwoo,zeesea,sace lady,kimuse,pink flash,colorkey,gilaa,bioderma,ordinary,annesa,paula choice,laneige,klair,simple,Dr.Jart+,estee lauder,skin1004,shisheido,Clé de Peau,chacot,obagi,clinique,missha,murad,cocoon,perfect uv gel,eucerin,cetaphil,neutrogena,hiruscar,srv,Dermarium


### 2. Unpivot the above table to create a brand/competitor

In [4]:
keyword_type_dict = []

for x in range(len(keyword_type_df_brand.columns)):
    keyword_type = keyword_type_df_brand.columns[x]
    for y in keyword_type_df_brand.iloc[0:, x]:  
        keyword = y
        brand = keyword_type_df_brand.iloc[0:, x][0]
        row = {
        'keyword': keyword,
        'keyword_type': keyword_type,
        'brand': brand}
        
        keyword_type_dict.append(row)
        
keyword_type_df_brand_final = pd.DataFrame(keyword_type_dict)
keyword_type_df_brand_final = keyword_type_df_brand_final.dropna(subset=['keyword']).drop_duplicates()
keyword_type_df_brand_final.head(2)

,keyword,keyword_type,brand
0,La Roche Posay,Branded,La Roche Posay
1,la roche posay,Branded,La Roche Posay


### 3. Combine the brand/competitor table with generic table

In [5]:
def clean_kw(kw): return kw.replace('\xa0', ' ').strip()
keyword_type_df = keyword_type_df_brand_final.copy()
keyword_type_df = keyword_type_df.astype(str)
keyword_type_df['keyword'] = keyword_type_df['keyword'].apply(clean_kw)
keyword_type_df['brand'] = keyword_type_df['brand'].apply(clean_kw)

In [6]:
column_delete = ['Skin Care','Makeup','Hair','General','Others','Fragrance.1']

def get_subcat_data(subcat):
    subcat_df_v2 = pd.read_excel("C:/Users/gianhubinh.le/OneDrive - L'Oréal/Documents - -VN- VNCDO - Data work-space/Media Dashboard/Other/EMT Keyword.xlsx", sheet_name = subcat, skiprows=1)
    columns_to_keep = [col for col in subcat_df_v2.columns if 'NOT' not in col]
    columns_to_keep = [col for col in columns_to_keep if col not in column_delete]
    subcat_df_v2 = subcat_df_v2[columns_to_keep].copy()

    subcat_dict = []
    for cols in subcat_df_v2.columns:
        for y in subcat_df_v2[cols]:
            if type(y) != float:
                row = {
                'keyword': y,
                'not_keyword': None,
                'sub_category': cols
                }
                subcat_dict.append(row)

    subcat_df = pd.DataFrame(subcat_dict)
    subcat_df = subcat_df.dropna(subset=['keyword']).drop_duplicates()
    return subcat_df

skincare_sub_lookup_df = get_subcat_data('Skin Care')
makeup_sub_lookup_df = get_subcat_data('Makeup')
hair_sub_lookup_df = get_subcat_data('Hair')
fragrance_sub_lookup_df = get_subcat_data('Fragrance')

skincare_sub_lookup_df = skincare_sub_lookup_df.drop_duplicates().astype(str)
makeup_sub_lookup_df = makeup_sub_lookup_df.drop_duplicates().astype(str)
hair_sub_lookup_df = hair_sub_lookup_df.drop_duplicates().astype(str)
fragrance_sub_lookup_df = fragrance_sub_lookup_df.drop_duplicates().astype(str)

In [7]:
keyword_type_df = keyword_type_df.drop_duplicates().astype(str)
brand_lst = set(keyword_type_df['brand'].unique())
brand_lst = {item for item in brand_lst if item != 'None'}

keywords_dict = {
    str(row.keyword) : row.keyword_type
    for row in keyword_type_df.itertuples()
    if row.keyword_type != 'Generic'
}

sub_category_dict = {}

skincare_sub_dict = {
    str(row.keyword) : [row.sub_category, 'Skin Care']
    for row in skincare_sub_lookup_df.itertuples() if (not(pd.isna(row.keyword)))
}
sub_category_dict.update(skincare_sub_dict)

makeup_sub_dict = {
    str(row.keyword) : [row.sub_category, 'Makeup']
    for row in makeup_sub_lookup_df.itertuples() if (not(pd.isna(row.keyword)))
}
sub_category_dict.update(makeup_sub_dict)

hair_sub_dict = {
    str(row.keyword) : [row.sub_category, 'Hair']
    for row in hair_sub_lookup_df.itertuples() if (not(pd.isna(row.keyword)))
}
sub_category_dict.update(hair_sub_dict)

fragrance_sub_dict = {
    str(row.keyword) : [row.sub_category, 'Fragrance']
    for row in fragrance_sub_lookup_df.itertuples() if (not(pd.isna(row.keyword)))
}
sub_category_dict.update(fragrance_sub_dict)

sub_to_cate = {}

skincare_sub_to_cate = {
    str(row.keyword) : 'Skin Care'
    for row in skincare_sub_lookup_df.itertuples() if (not(pd.isna(row.keyword)))
}
sub_to_cate.update(skincare_sub_to_cate)

makeup_sub_to_cate = {
    str(row.keyword) : 'Makeup'
    for row in makeup_sub_lookup_df.itertuples() if (not(pd.isna(row.keyword)))
}
sub_to_cate.update(makeup_sub_to_cate)

hair_sub_to_cate = {
    str(row.keyword) : 'Hair'
    for row in hair_sub_lookup_df.itertuples() if (not(pd.isna(row.keyword)))
}
sub_to_cate.update(hair_sub_to_cate)

fragrance_sub_to_cate = {
    str(row.keyword) : 'Fragrance'
    for row in fragrance_sub_lookup_df.itertuples() if (not(pd.isna(row.keyword)))
}
sub_to_cate.update(fragrance_sub_to_cate)

category_not_dict = {}

# for row in category_lookup_df.itertuples():
#     if (not(pd.isna(row.not_keyword))):
#         if row.not_keyword in category_not_dict.keys():
#             category_not_dict[row.not_keyword].append(row.category)
#         else:
#             category_not_dict[row.not_keyword] = [row.category]

In [8]:
def predict_brand_reverse_fuzzy(keyword):
    keyword_split = str.split(keyword)
    keyword_split.append(keyword)

    brand_similarity_dict = {
        "keyword_split": None,
        "brand": None,
        "brand_match_type" : None,
        "score": 0
    }

    for kw in keyword_split:
        for brand in brand_lst:
            score = round(Levenshtein.ratio(brand.lower(),kw.lower()),2)
            if score >= brand_similarity_dict['score']:
                brand_similarity_dict['keyword_split'] = kw
                brand_similarity_dict['brand'] = brand
                brand_similarity_dict['score'] = score
                try:
                    brand_similarity_dict['brand_match_type'] = keywords_dict[brand]
                except:
                    brand_similarity_dict['brand_match_type'] = None

    return brand_similarity_dict

predict_brand_reverse_fuzzy("jbhk")

{'keyword_split': 'jbhk',
 'brand': 'Black Rouge',
 'brand_match_type': 'Competitor',
 'score': 0.27}

In [9]:
def predict(keyword):
    
    threshold1 = 75 #keyword type fuzzy match score between 55 inclusive - 85 exclusive would be Combined (below 55 will be Generic)
    threshold2 = 85 #keywrod type fuzzy match score between 85 inclusive - 100 would be Branded or Competitor
    threshold3 = 70 #subcategory fuzzy match score below 60 would be considered as Undefined
    threshold4 = 0.8 #keywrod type fuzzy match score between 80 and 100 would be branded / competitor, possibly combined
        
    row = {
        'keywords': keyword,
        'keyword_type': None,
        'brand': None,
        'keyword_category': None,
        'keyword_subcategory': []
    }

    row['keyword_sub_categories'] = []
    row['keyword_type'] = 'Generic' #default the keyword type to Generic
    row['keyword_sub_categories'] = [sub_category for keyword_subcat, sub_category in sub_category_dict.items() if str(row[r'keywords']).casefold() == str(keyword_subcat).casefold()]        
    
    # 1. Branded score reverse
    brand_match_raw = predict_brand_reverse_fuzzy(keyword)
    brand_match = brand_match_raw['brand']
    brand_match_score_reverse = brand_match_raw['score']
    brand_match_type = brand_match_raw['brand_match_type']
    # 2. Branded score
    types = findMatchKey(keyword, list(keywords_dict.keys())) # token sort best match for keyword type
    types_tokenset = findMatchKey2(str(keyword), list(keywords_dict.keys()))
    
    # ------------------------------------------------I. BRANDED (TYPE)----------------------------------------------------------------#
    if types[0][1] >= threshold2:
        row['keyword_type'] = keywords_dict[types[0][0]]
        brand = keyword_type_df[keyword_type_df['keyword'] == types[0][0]].reset_index(drop=True).iloc[0, 2]
        row['brand'] = brand
    elif types_tokenset[0][1]>=threshold1 and " " in str(keyword):  #single word should not be combined keywords
        row['keyword_type'] = keywords_dict[types_tokenset[0][0]] + ' Combined'
        brand = keyword_type_df[keyword_type_df['keyword'] == types_tokenset[0][0]].reset_index(drop=True).iloc[0, 2]
        row['brand'] = brand
        
    # ------------------------------------------------II. CAT & SUBCAT----------------------------------------------------------------#
    if row['keyword_type'] == 'Branded' or row['keyword_type'] == 'Competitor':
        row['keyword_category'] = 'NA'
        row['keyword_subcategory'] = 'NA'
    else:  
        if len(row['keyword_sub_categories'])>0:
            row['keyword_subcategory'] = row['keyword_sub_categories'][0][0]
            row['keyword_category'] = row['keyword_sub_categories'][0][1]

        else:
            subcat_score = findMatchKey2(row['keywords'], list(sub_category_dict.keys()))[0][1]
            if keyword in sub_category_dict:
                sub_cates = sub_category_dict[row['keywords']]
            # các keyword có số ký tự >10 có thể nới lỏng điểm 1 chút, tuy nhiên keyword ngắn thì phải có độ tương đồng cao
            elif ((subcat_score >= threshold3 and len(keyword) >= 10) or (subcat_score >= threshold3 + 20 and len(keyword) >= 4)):
                sub_cates = findMatchKey2(str(keyword), list(sub_category_dict.keys()))
                row['keyword_subcategory'] = sub_category_dict[sub_cates[0][0]][0]
                row['keyword_category'] = sub_category_dict[sub_cates[0][0]][1]
            else:
                row['keyword_subcategory'] = 'Undefined'
                row['keyword_category'] = 'Undefined'
    
    # # ------------------------------------------------III. Reverse prediction--------------------------------------------------
    if row['keyword_type'] == 'Generic' and (row['keyword_category'] == 'Undefined' or row['keyword_category'] == 'NA'):
        if brand_match_score_reverse >= threshold4:
            row['brand'] = brand_match
            row['keyword_type'] = brand_match_type
    elif row['keyword_type'] == 'Generic' and (row['keyword_category'] != 'Undefined' and row['keyword_category'] != 'NA'):
        if brand_match_score_reverse >= threshold4:
            row['brand'] = brand_match
            row['keyword_type'] = brand_match_type + ' Combined'
    
    # print(findMatchKey2(row['keywords'], list(sub_category_dict.keys())))
    # print(subcat_score)
    return  pd.DataFrame([row])


# chì kẻ mày chống nước
# dưỡng môi pcd chính hãng
predict('nuoc hoa')

,keywords,keyword_type,brand,keyword_category,keyword_subcategory,keyword_sub_categories
0,nuoc hoa,Generic,None,Fragrance,Fragrance,"[[Fragrance, Fragrance]]"


In [10]:
df_result_lst = []

for kw in tqdm(df_keyword['keyword'], desc="Processing Keywords"):
# for kw in df_keyword['keyword']:
    df_result_row = predict(kw)
    df_result_lst.append(df_result_row)

Processing Keywords: 100%|███████████████████████████████████████████████████| 143353/143353 [1:01:06<00:00, 39.10it/s]


In [11]:
df_result = pd.concat(df_result_lst, ignore_index=True)
df_result.head(2)
df_result = df_result.astype(str)

project_id = "spmena-sandbox-vn-apac-dv"
table_id = 'vn_search_dashboard.search_keywords_dim'
pandas_gbq.to_gbq(df_result, table_id, project_id=project_id, if_exists = 'replace')

sql = """
create or replace table `spmena-sandbox-vn-apac-dv.vn_search_dashboard.search_keywords_dim` as
SELECT distinct
lower(NORMALIZE(keywords, NFC)) as keywords, * except(keywords)
FROM `spmena-sandbox-vn-apac-dv.vn_search_dashboard.search_keywords_dim`
order by keywords asc
"""
pd.read_gbq(sql,project_id=project_id, use_bqstorage_api=True)  
print('Done updating')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


Done updating


In [12]:
df_result = pd.concat(df_result_lst, ignore_index=True)
df_result.head(2)

,keywords,keyword_type,brand,keyword_category,keyword_subcategory,keyword_sub_categories
0,bộ sản phẩm trị mụn la roche posay,Branded Combined,La Roche Posay,Skin Care,Spots & Blemishes,[]
1,toner skin1004,Competitor Combined,Skin1004,Skin Care,Toner & Mists,[]
